In [26]:
import tifffile
import numpy as np
import skimage.filters as filters
import napari

In [27]:
%gui qt

In [28]:
path = "/mnt/5404b8a5-71b7-4464-9a1e-b40cd26fac58/Data_Drive/TRPC6 Spacetraining Data/TRPC6 GSK Spacetraining/190131/N1A5/000_D_190130_A_N1A5.tif"

In [34]:
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()

file_path = filedialog.askopenfilename()

In [35]:
volume = tifffile.imread(file_path)

In [36]:
import napari
import skimage.data
import skimage.filters
from napari.types import ImageData, LabelsData

from magicgui import magicgui

def threshold_image(image, percent):
    temp_img = image.copy()
    temp_img[temp_img<np.max(temp_img)*percent]=0
    return temp_img

def binary_labels(image):
    temp_img = image.copy()
    temp_img[temp_img >0]=1
    return temp_img
# - `auto_call` tells magicgui to call the function when a parameter changes
# - we use `widget_type` to override the default "float" widget on sigma,
#   and provide a maximum valid value.
# - we contstrain the possible choices for `mode`
@magicgui(
    auto_call=True,
    percent={"widget_type": "FloatSlider", "max": 1},
    layout='horizontal'
)
def precent_threshold(layer: ImageData, percent: float = .4) -> ImageData:
    """Apply a precent threshold ``layer``."""
    if layer is not None:
        label_img =  threshold_image(layer, percent=percent)
        return label_img

@magicgui(call_button = 'Generate Label', )
def label_gen(layer: ImageData,) -> LabelsData:
    if layer is not None:
        label_img =  binary_labels(layer)
        return label_img
# create a viewer and add some images
viewer = napari.Viewer()
viewer.add_image(volume, name="neuron")

# Add it to the napari viewer
viewer.window.add_dock_widget(precent_threshold)
viewer.window.add_dock_widget(label_gen)
#viewer.window.add_dock_widget(label_gen)
# update the layer dropdown menu when the layer list changes
viewer.layers.events.changed.connect(precent_threshold.reset_choices)

napari.run()

In [8]:
viewer = napari.Viewer()
